In [2]:
import numpy as np # linear algebra
import pandas as pd # pandas for dataframe based data processing and CSV file I/O
import requests # for http requests
from bs4 import BeautifulSoup # for html parsing and scraping
import bs4

import matplotlib.pyplot as plt
import seaborn as sns
import json
#from tidylib import tidy_document # for tidying incorrect html

sns.set_style('whitegrid')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
content = '''<div class="box">
   <div class="css-a9jdxq-Container e1nzkzy80">
      <h2 class="css-1dd55xt-Title e1hnx4sx0">Documents</h2>
      <input class="css-eo3c1x-Input-input ep3169p0" 
	placeholder="Documents" 
	aria-label="doctor-search" 
	value="" 
	style="margin: 0px 0px 8px;">
      
   </div>
</div>'''
js = HTMLtoJSONParser.to_json(content)
print (js)


In [2]:
import html
class HTMLtoJSONParser(html.parser.HTMLParser):
    def __init__(self, raise_exception = True) :
        html.parser.HTMLParser.__init__(self)
        self.doc  = { }
        self.path = []
        self.cur  = self.doc
        self.line = 0
        self.raise_exception = raise_exception
         
    @property
    def json(self):
        return self.doc
         
    @staticmethod
    def to_json(content, raise_exception = True):
        parser = HTMLtoJSONParser(raise_exception = raise_exception)
        parser.feed(content)
        return parser.json
         
    def handle_starttag(self, tag, attrs):
        self.path.append(tag)
        attrs = { k:v for k,v in attrs }
        if tag in self.cur :
            if isinstance(self.cur[tag],list) :
                self.cur[tag].append(  { "__parent__": self.cur } )
                self.cur = self.cur[tag][-1]
            else :
                self.cur[tag] = [ self.cur[tag] ]
                self.cur[tag].append(  { "__parent__": self.cur } )
                self.cur = self.cur[tag][-1]
        else :
            self.cur[tag] = { "__parent__": self.cur }
            self.cur = self.cur[tag]
             
        for a,v in attrs.items():
            self.cur["#" + a] = v
        self.cur[""] = ""
                 
    def handle_endtag(self, tag):
        if tag != self.path[-1] and self.raise_exception :
            raise Exception("html is malformed around line: {0} (it might be because of a tag <br>, <hr>, <img .. > not closed)".format(self.line))
        del self.path[-1]
        memo = self.cur
        self.cur = self.cur["__parent__"]
        self.clean(memo)
                 
    def handle_data(self, data):
        self.line += data.count("\n")
        if "" in self.cur :
            self.cur[""] += data
             
    def clean(self, values):
        keys = list(values.keys())
        for k in keys:
            v = values[k]
            if isinstance(v, str) :
                #print ("clean", k,[v])
                c = v.strip(" \n\r\t")
                if c != v : 
                    if len(c) > 0 : 
                        values[k] = c
                    else : 
                        del values[k]
        del values["__parent__"]

In [ ]:
dic

In [9]:
import codecs
f=codecs.open("index.html", 'r', 'utf-8')

from bs4 import BeautifulSoup

dic = dict()


itt = 0


def list_tree_names(node):
    global itt
    for child in node.contents:
        try:
            dic.update({child.name + "/" + str(itt): child.attrs})
            itt += 1
            list_tree_names(node=child)
        except:
            dic.update({"text" + "/" + str(itt): child})
            itt += 1
            
            

soup = BeautifulSoup(f.read(), "html.parser")

list_tree_names(soup.find("body"))

# for item in soup.contents:
#     a = item
#     dic.update({item.name + str(itt): item.attrs})
#     itt += 1



In [ ]:
dic

In [ ]:
import xmltojson
import json
import requests
# Sample URL to fetch the html page
url = "https://geeksforgeeks.com"
  
# Headers to mimic the browser
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
    (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
}
  
# Get the page through get() method
html_response = requests.get(url=url, headers = headers)
  
# Save the page content as sample.html
with open("sample.html", "w") as html_file:
    html_file.write(html_response.text)

In [11]:
import codecs
f=codecs.open("DOM.html", 'r', 'utf-8')
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)

    def handle_endtag(self, tag):
        print("End tag :", tag)

    def handle_data(self, data):
        print("Data  :", data)  

parser = MyHTMLParser()
#parser.feed(f.read())

In [ ]:

with open('master.html', 'r') as f:

    contents = f.read()

    soup = BeautifulSoup(contents, 'lxml')

    for child in soup.recursiveChildGenerator():

        if child.name:

            print(child.name)
        

In [ ]:
import codecs
import csv
from bs4 import BeautifulSoup # for html parsing and scraping
f=codecs.open("master.html", 'r', 'utf-8')
#document= BeautifulSoup(f.read(), "html5lib")
#print(document.prettify()) 
soup = BeautifulSoup(f.read(), 'html5lib') 
tags=["a","abbr","acronym","address","area","bdo","big","blockquote","br","button","caption","cite","code","col","colgroup","dd","del","dfn","dl","DOCTYPE","dt","em","fieldset","form","h1","h2","h3","h4","h5","h6","hr","i","img","input","ins","kbd","label","legend","li","map","meta","noscript","object","ol","optgroup","option","p","param","pre","q","samp","select","small","strong","sub","sup","table","tbody","td","textarea","tfoot","th","thead","title","tr","tt","ul","var"]
with open('htmlcsv.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Tag_Name", "Text","Attributes"])
            for i in tags:
                #print("Tag:"+i)
                for j in (soup.findAll(i)):
                    print(j)
                    #print("Text:"+j.get_text())
                    #print("Atrributes:"+str(j.attrs))

                    writer.writerow([str(i),str(j.get_text()),str(j.attrs)])





In [ ]:
!pip install --upgrade lxml

In [ ]:
import selenium.webdriver as webdriver
import lxml.html as lh
import lxml.html.clean as clean

browser = webdriver.Chrome() # Get local session of Chrome
browser.get("http://localhost:3000") # Load page

content=browser.page_source
cleaner=clean.Cleaner()
content=cleaner.clean_html(content) 
doc=lh.fromstring(content)

In [18]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

BASE_URL = "https://www.awesome-nlp.com"



def get_category_links(section_url):
    # Put the stuff you see when using Inspect Element in a variable called html.
    html = urlopen(section_url).read()    
    # Parse the stuff.
    soup = BeautifulSoup(html, "lxml")    
    # The next two lines will change depending on what you're looking for. This 
    # line is looking for <dl class="boccat">.  
    boccat = soup.find("dl", "boccat")    
    # This line organizes what is found in the above line into a list of 
    # hrefs (i.e. links). 
    category_links = [BASE_URL + dd.a["href"] for dd in boccat.findAll("dd")]
    return category_links